In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
# quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
trim_cd8_na <- readRDS(file = 'trim_cd8_na.rds')

# CD8 Naive Gate Plot

In [ ]:
cd8 <- readRDS(file = '../../Seurat_Objects/cd8_subset.rds')
cd8.batch <- SplitObject(cd8, split.by = 'batch_id')
cd8.batch <- lapply(cd8.batch, function(x){
    NormalizeData(x, assay = 'ADT', normalization.method = 'CLR', margin = 2)})

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeatureScatter(object = cd8.batch$B069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id', cols = 'white', pt.size = 0) & geom_density_2d(size = 1) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeatureScatter(object = cd8.batch$B069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id', cols = 'white', pt.size = 0) & geom_density_2d(size = 1, bins = 30) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
FeatureScatter(object = cd8.batch$B069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id', cols = 'black', pt.size = 0) & geom_density_2d(size = 1, binwidth = 0.075) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')

In [ ]:
# Open a pdf file
pdf("plots/cd8na_gating.pdf", width = 8, height = 8) 
# 2. Create a plot
FeatureScatter(object = cd8.batch$B069, feature1 = 'adt_CD27', feature2 = 'adt_CD45RA', group.by = 'batch_id', cols = 'white', pt.size = 0) & geom_density_2d(size = 1) &
geom_vline(xintercept = 2.5) & geom_hline(yintercept = 3.1) & ggtitle('B069')
# Close the pdf file
dev.off() 

# 3WNN

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)
DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'pure_celltype', label = FALSE, label.size = 8, repel = TRUE, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d'))

In [ ]:
# Open a pdf file
pdf("plots/cd8na_celltypes_3WNN_umap.pdf", width = 12, height = 10) 
# 2. Create a plot
DimPlot(trim_cd8_na, reduction = 'wnn.3.umap', group.by = 'pure_celltype', label = FALSE, label.size = 8, repel = TRUE, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d'))
# Close the pdf file
dev.off() 

# Heatmap

In [ ]:
marker_genes_rna <- c('CCR7','CD27','FAS')
marker_avg_rna <- AverageExpression(trim_cd8_na, features = marker_genes_rna, assay = 'SCT')
marker_genes_adt <- c('CD45RA','CD197','CD27','CD49d','CD161','TCR-Valpha7.2')
marker_avg_adt <- AverageExpression(trim_cd8_na, features = marker_genes_adt, assay = 'clean_adt')

In [ ]:
library(pheatmap)

In [ ]:
marker_avg_rna <- marker_avg_rna$SCT
# marker_avg_rna

In [ ]:
marker_avg_adt <- marker_avg_adt$clean_adt
# marker_avg_adt

In [ ]:
rownames(marker_avg_adt) <- paste0('adt_',rownames(marker_avg_adt))

In [ ]:
# marker_avg_rna
# marker_avg_adt

In [ ]:
markers_bind <- rbind(marker_avg_rna,marker_avg_adt)
markers_bind <- as.data.frame(markers_bind)
markers_bind

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
cd8na_markers_heatmap <- pheatmap(markers_bind, scale = 'row', fontsize = 20)

# Raindrop plots

In [ ]:
sp_merge <- readRDS(file = '../../Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd8_nonnaive <- subset(sp_merge, idents = c('CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA'))
cd8_nonnaive$pure_celltype <- cd8_nonnaive$gating_celltype

In [ ]:
cd8_merge <- merge(trim_cd8_na, cd8_nonnaive)

In [ ]:
cd8_merge <- NormalizeData(cd8_merge, assay = 'RNA')
cd8_merge <- NormalizeData(cd8_merge, assay = 'ADT', margin = 2, normalization.method = 'CLR')
cd8_merge <- SetIdent(cd8_merge, value = 'pure_celltype')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 20)
RidgePlot(cd8_merge, features = c('adt_CD27','rna_CD27',
                                  'adt_CD197','rna_CCR7',
                                  'adt_CD45RA','adt_CD49d',
                                  'adt_CD161','adt_TCR-Valpha7.2'), group.by = 'pure_celltype', ncol = 2)

In [ ]:
df_build <- function(seurat_object){
    meta <- seurat_object@meta.data
    meta$TCRValpha7.2 <- as.vector(seurat_object@assays$ADT['TCR-Valpha7.2',])
    meta$CD161 <- as.vector(seurat_object@assays$ADT['CD161',])
    meta$CD197 <- as.vector(seurat_object@assays$ADT['CD197',])
    meta$CD45RA <- as.vector(seurat_object@assays$ADT['CD45RA',])
    meta$CD27 <- as.vector(seurat_object@assays$ADT['CD27',])
    return(meta)}

In [ ]:
test_df <- df_build(cd8_merge)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
ggplot(test_df, aes(x = pure_celltype, y = CD45RA, color=pure_celltype)) + 
  ggdist::stat_halfeye(
    adjust = .5, 
    width = .6, 
    .width = 0, 
    justification = -.2, 
    point_colour = NA
  ) + 
  geom_boxplot(
    width = .15, 
    outlier.shape = NA
  ) +
  ## add justified jitter from the {gghalves} package
  gghalves::geom_half_point(
    ## draw jitter on the left
    side = "l", 
    ## control range of jitter
    range_scale = .4, 
    ## add some transparency
    alpha = .3
  ) +
  coord_cartesian(xlim = c(1.2, NA), clip = "off")

## UMAP Plots

In [ ]:
library(cowplot)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
# p1 <- FeaturePlot(trim_cd8_na, features = 'cleanadt_CD27', reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis(limits=c(1,4.5))
# p2 <- FeaturePlot(trim_cd8_na, features = 'cleanadt_CD197', reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis(limits=c(1,1.75))
# p3 <- FeaturePlot(trim_cd8_na, features = 'cleanadt_CD45RA', reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis(limits=c(2,5))
# p3 <- FeaturePlot(trim_cd8_na, features = 'cleanadt_TCR-Valpha7.2', reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis()
# p1 <- FeaturePlot(trim_cd8_na, features = 'cleanadt_CD49d', reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis()
# FeaturePlot(trim_cd8_na, features = 'sct_FAS', reduction = 'wnn.3.umap', order = TRUE, pt.size = 1) & scale_color_viridis()

In [ ]:
p1 <- plot_density(trim_cd8_na, features = 'cleanadt_CD49d', reduction = 'wnn.3.umap', pal = 'inferno')
p2 <- plot_density(trim_cd8_na, features = 'sct_FAS', reduction = 'wnn.3.umap')
p3 <- plot_density(trim_cd8_na, features = 'cleanadt_TCR-Valpha7.2', reduction = 'wnn.3.umap', pal = 'inferno')
p4 <- plot_density(trim_cd8_na, features = 'sct_IFNG', reduction = 'wnn.3.umap')
# p5 <- plot_density(trim_cd8_na, features = 'cleanadt_CD49d', reduction = 'wnn.3.umap')
# p6 <- plot_density(trim_cd8_na, features = 'sct_FAS', reduction = 'wnn.3.umap', pal = 'inferno')

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 14)
plot_grid(p1, p2, p3, p4, ncol = 2)

In [ ]:
# Open a pdf file
pdf("plots/cd8na_celltypes_markers_umap.pdf", width = 14, height = 14) 
# 2. Create a plot
plot_grid(p1, p2, p3, p4, ncol = 2)
# Close the pdf file
dev.off() 

In [ ]:
genes <- c('cleanadt_CD45RA','cleanadt_CD197','cleanadt_CD27','sct_LEF1','sct_BACH2','cleanadt_CD49d','cleanadt_TCR-Valpha7.2','sct_FAS')

In [ ]:
trim_cd8_na <- NormalizeData(trim_cd8_na, assay = 'RNA')

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 14)
p1 <- VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_CD45RA', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(0,5) & coord_flip() & NoLegend()
p2 <- VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_CD197', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(0,1.75) & coord_flip() & NoLegend() & 
            theme(axis.title.y = element_blank(), axis.text.y = element_blank())
p3 <- VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_CD27', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(0,4.5) & coord_flip() & NoLegend() & 
            theme(axis.title.y = element_blank(), axis.text.y = element_blank())
p4 <- VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_CD49d', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(1,4.5) & coord_flip() & NoLegend() & 
            theme(axis.title.y = element_blank(), axis.text.y = element_blank())
# VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_CD95', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(0.5,2) & coord_flip() & NoLegend()
p5 <- VlnPlot(trim_cd8_na, group.by = 'pure_celltype', features = 'cleanadt_TCR-Valpha7.2', pt.size = 0, cols = c('#807dba','#01665e','#1d91c0','#08519c','#3f007d')) & ylim(0,3) & coord_flip() & NoLegend() & 
            theme(axis.title.y = element_blank(), axis.text.y = element_blank())

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 14)
plot_grid(p1, p2, p3, p4, p5, nrow = 1, rel_widths = c(1.3,1,1,1,1))

# Tracks

Check ATAC notebook 03-R_CD8_Na_ATAC.ipynb

# Subset Counts

In [ ]:
split_cd8_na <- SplitObject(trim_cd8_na, split.by = 'pediatric_senior')

In [ ]:
celltype_df_func <- function(seurat_object, metadata_slot, id_list){
    seurat_object <- SetIdent(seurat_object, value = metadata_slot)
    subset_object <- subset(seurat_object, idents = id_list[[1]])
    df <- as.data.frame(table(subset_object$pbmc_sample_id))
    colnames(df) <- c('sample',id_list[[1]])
    i = 2
    while(i < length(id_list) + 1){
        subset_object <- subset(seurat_object, idents = id_list[[i]])
        df2 <- as.data.frame(table(subset_object$pbmc_sample_id))
        df[,ncol(df) + 1] <- df2$Freq
        colnames(df)[ncol(df)] <- paste0(id_list[[i]])
        #df$id_list[[i]] <- df2$Freq
        i = i + 1
        }
    df
    }

In [ ]:
ped_celltype <- celltype_df_func(split_cd8_na$Pediatric, 'pure_celltype', sort(unique(split_cd8_na$Pediatric$pure_celltype)))

In [ ]:
head(ped_celltype)

In [ ]:
rownames(ped_celltype) <- ped_celltype$sample
ped_celltype <- ped_celltype[,2:6]
colnames(ped_celltype) <- sort(unique(split_cd8_na$Pediatric$pure_celltype))
ped_celltype

In [ ]:
sen_celltype <- celltype_df_func(split_cd8_na$Senior, 'pure_celltype', sort(unique(split_cd8_na$Senior$pure_celltype)))

In [ ]:
head(sen_celltype)

In [ ]:
rownames(sen_celltype) <- sen_celltype$sample
sen_celltype <- sen_celltype[,2:6]
colnames(sen_celltype) <- sort(unique(split_cd8_na$Senior$pure_celltype))
sen_celltype

In [ ]:
write.csv(ped_celltype, file = 'cd8na_freq_ped.csv')
write.csv(sen_celltype, file = 'cd8na_freq_OA.csv')

# Split Global UMAP

In [ ]:
sp_merge <- readRDS(file = '../../Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge$pure_celltype <- sp_merge$gating_celltype

## Add CD8na subsets

In [ ]:
cd8na_meta <- trim_cd8_na@meta.data

In [ ]:
meta_subset_cd8na <- sp_merge@meta.data[which(rownames(sp_merge@meta.data) %in% rownames(cd8na_meta)),]
length(rownames(meta_subset_cd8na))
table(rownames(meta_subset_cd8na) %in% rownames(cd8na_meta))
table(rownames(meta_subset_cd8na) == rownames(cd8na_meta))

Again, it's in the same order, so we don't need to reorder the metadata

In [ ]:
sp_merge$pure_celltype[which(rownames(sp_merge@meta.data) %in% rownames(cd8na_meta))] <- as.character(cd8na_meta$pure_celltype)

## Plot

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 12)
DimPlot(sp_merge, reduction = 'wnn.3.umap', group.by = 'pure_celltype', raster = FALSE, shuffle = TRUE, split.by = 'pediatric_senior', pt.size = 0.1,
        order = c('CD8 MAIT','CD8 MNP-1','CD8 Naive','CD8 MNP-2','CD8 SCM',
                  'CD4 CM','CD4 EM1','CD4 EM2','CD4 Naive',
                  'CD4 TEMRA','CD4 Unk','CD8 CM','CD8 EM1','CD8 EM2',
                  'CD8 TEMRA','CD8 Unk','Treg'),
        cols = c('lightgrey','lightgrey','lightgrey',
                 'lightgrey','lightgrey','lightgrey','lightgrey','lightgrey',
                 'lightgrey','lightgrey','lightgrey','lightgrey',
                 '#1d91c0','#08519c','#807dba','#01665e','#3f007d'))

In [ ]:
# Open a pdf file
pdf("plots/cd8na_celltypes_global_umap.pdf", width = 25, height = 12) 
# 2. Create a plot
DimPlot(sp_merge, reduction = 'wnn.3.umap', group.by = 'pure_celltype', raster = FALSE, shuffle = TRUE, split.by = 'pediatric_senior', pt.size = 0.1,
        order = c('CD8 MAIT','CD8 MNP-1','CD8 Naive','CD8 MNP-2','CD8 SCM',
                  'CD4 CM','CD4 EM1','CD4 EM2','CD4 Naive',
                  'CD4 TEMRA','CD4 Unk','CD8 CM','CD8 EM1','CD8 EM2',
                  'CD8 TEMRA','CD8 Unk','Treg'),
        cols = c('lightgrey','lightgrey','lightgrey',
                 'lightgrey','lightgrey','lightgrey','lightgrey','lightgrey',
                 'lightgrey','lightgrey','lightgrey','lightgrey',
                 '#1d91c0','#08519c','#807dba','#01665e','#3f007d'))
# Close the pdf file
dev.off() 

# Gene Expression DotPlot

In [ ]:
gene_list_updated <- c('ITGA6','CD55','ITGB1','ITGA4','KLRD1','ITGAL','KLRC2','KLRC3','IL21R','KLRB1','IL12RB2','KLRG1','LEF1','BACH2','FOXP1','RUNX2','BCL2',
                       'TOX','IKZF2','RUNX3','JUNB','RORA','IFNG-AS1','GZMK','GZMA','PRF1','NCAM1','PECAM1')

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'pure_celltype')
levels(trim_cd8_na) <- c('CD8 MAIT','CD8 SCM','CD8 MNP-2','CD8 MNP-1','CD8 Naive')
levels(trim_cd8_na)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 4)
DotPlot(trim_cd8_na, features = gene_list_updated, 
        col.min = 0, assay = 'SCT') & theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 20),
                                            axis.text.y = element_text(size = 20),
                                            axis.title.x = element_blank(), axis.title.y = element_blank())

In [ ]:
# Open a pdf file
pdf("plots/cd8na_celltypes_RNA_dotplot.pdf", width = 12, height = 4) 
# 2. Create a plot
DotPlot(trim_cd8_na, features = gene_list_updated, 
        col.min = 0, assay = 'SCT') & theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 20),
                                            axis.text.y = element_text(size = 20),
                                            axis.title.x = element_blank(), axis.title.y = element_blank())
# Close the pdf file
dev.off() 

In [ ]:
'#c7e9c0','#74c476','#238b45','#00441b','#6baed6','#02818a','#08519c','#6a51a3','#bdbdbd','#737373'

In [ ]:
'#bcbddc','#807dba','#6a51a3','#54278f','#bdbdbd','#08519c','#969696','#ae017e','#3f007d'

# DEGs

In [ ]:
trim_cd8_na <- SetIdent(trim_cd8_na, value = 'pure_celltype')
degs <- FindAllMarkers(trim_cd8_na, assay = 'SCT')

In [ ]:
head(degs)

In [ ]:
write.csv(degs, file = 'cd8na_degs.csv')